In [1]:
import re
from dataclasses import dataclass, field
from functools import cache
from typing import NamedTuple
from __future__ import annotations


class Resource(NamedTuple):
    geode: int
    obsidian: int
    clay: int
    ore: int
    
    @cache
    def __gt__(self, resource: Resource):
        return all(r1 > r2 for r1, r2 in zip(self, resource))
    
    @cache
    def __lt__(self, resource: Resource):
        return any(r1 < r2 for r1, r2 in zip(self, resource))
    
    @cache
    def __ge__(self, resource: Resource):
        return all(r1 >= r2 for r1, r2 in zip(self, resource))
    
    @cache
    def __le__(self, resource: Resource):
        return any(r1 <= r2 for r1, r2 in zip(self, resource))
    
    @cache
    def __add__(self, resource: Resource):
        return Resource(*(r1 + r2 for r1, r2 in zip(self, resource)))
    
    @cache
    def __sub__(self, resource: Resource):
        return Resource(*(r1 - r2 for r1, r2 in zip(self, resource)))

@dataclass
class Robot:
    name: str
    costs: Resource
    prods: Resource = field(repr=False)

@dataclass
class Blueprint:
    number: int
    robots: tuple[Robot, ...]

blueprints = {}
with open("Day19.txt") as file:
    for line in file:
        i, a, b, c, d, e, f = map(int, re.findall(r"\d+", line))
        blueprint = Blueprint(i, (
            Robot("Geode", Resource(0, f, 0, e), Resource(1, 0, 0, 0)), 
            Robot("Obsidian", Resource(0, 0, d, c), Resource(0, 1, 0, 0)), 
            Robot("Clay", Resource(0, 0, 0, b), Resource(0, 0, 1, 0)), 
            Robot("Ore", Resource(0, 0, 0, a), Resource(0, 0, 0, 1)), 
            Robot("None", Resource(0, 0, 0, 0), Resource(0, 0, 0, 0))
        ))
        blueprints[i] = blueprint

In [2]:
def run(blueprint, time, sort_key=None, threshold=100):
    sort_key = sort_key or (lambda e: tuple(zip(*e)))
    todo = [(Resource(0, 0, 0, 0), Resource(0, 0, 0, 1))]
    for minute in range(time):
        temp = []
        for having, making in todo:
            for robot in blueprint.robots:
                if having < robot.costs:
                    continue
                have, make = (having + making - robot.costs), (making + robot.prods)
                temp.append((have, make))
        todo = sorted(temp, reverse=True, key=sort_key)[:threshold]
    return max(having.geode for having, making in todo)

In [3]:
%%time
sum(run(blueprint, 24, threshold=200) * i for i, blueprint in blueprints.items())

CPU times: user 1.29 s, sys: 13.2 ms, total: 1.3 s
Wall time: 1.31 s


1650

In [4]:
%%time
from math import prod
sort_key = lambda e: tuple(e[1])
prod(run(blueprint, 32, sort_key=sort_key) for i, blueprint in blueprints.items() if i < 4)

CPU times: user 68.8 ms, sys: 3.02 ms, total: 71.8 ms
Wall time: 71.2 ms


5824